# Analizador de sentimiento

Descripción:
- Importar librerías y descargar recursos: Se importan las librerías necesarias y se descarga el lexicón de VADER para el análisis de sentimientos.
- Cargar y limpiar datos: Se carga el dataset y se filtran las sinopsis para eliminar las que no contienen información útil.
- Preprocesamiento del texto: Se aplica una función de preprocesamiento a las sinopsis para dejar solo palabras alfabéticas en minúsculas.
- Análisis de sentimientos: Se utiliza VADER para determinar el sentimiento de cada sinopsis y se clasifica como 'feliz', 'triste' o 'neutral'.
-Vectorización con TF-IDF: Se transforman las sinopsis en representaciones TF-IDF.
- Modelo KNN: Se entrena un modelo KNN para encontrar animes similares.
- Guardar y cargar modelos: Se guardan el modelo KNN y el vectorizador TF-IDF, y luego se cargan para su uso posterior.
- Recomendación de animes: Se define una función para recomendar animes basados en el estado de ánimo proporcionado, utilizando el modelo KNN entrenado.

## Librerías

In [4]:
import pandas as pd
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import joblib
import nltk

## Preparación de los datos y bel entrenamiento del modelo

In [6]:
# Descargar los recursos necesarios para VADER
nltk.download('vader_lexicon')

# Cargar el dataset
df = pd.read_csv("/content/drive/MyDrive/dataset/tfm/scraping_animeList.csv")

# Filtrar sinopsis con información y eliminar filas con sinopsis nulos
df = df.dropna(subset=['Synopsis'])
df = df[~df['Synopsis'].str.contains('No synopsis information')]

"""
  Limpiar el texto de la sinopsis dejando solo palabras alfabéticas
  y convirtiendo todo el texto a minúsculas.
"""
def preprocess_text(text):
    text = ' '.join(re.findall(r'\b[a-zA-Z]+\b', str(text)))
    text = re.sub(r'\s+', ' ', text).strip()
    return text.lower()

# Aplicar preprocesamiento al texto de las sinopsis
df['Synopsis'] = df['Synopsis'].apply(preprocess_text)

# Análisis de sentimientos con VADER
sid = SentimentIntensityAnalyzer()

"""
  Determinar el sentimiento del texto usando VADER y Clasificarlo
  como 'feliz', 'triste' o 'neutral' basado en el puntaje compuesto.
"""
def obtener_animo(text):
    scores = sid.polarity_scores(text)
    if scores['compound'] >= 0.5:
        return 'feliz'
    elif scores['compound'] <= -0.5:
        return 'triste'
    else:
        return 'neutral'

# Aplicar análisis de sentimientos a las sinopsis
df['Animo'] = df['Synopsis'].apply(obtener_animo)

# Vectorizador TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8, min_df=0.001)

# Transformar sinopsis a representaciones TF-IDF
X_tfidf = vectorizer.fit_transform(df['Synopsis'])

# Modelo KNN
modelo_knn = NearestNeighbors(n_neighbors=5, metric='cosine').fit(X_tfidf)

# Guardar el modelo y el vectorizador
joblib.dump(modelo_knn, '/content/drive/MyDrive/dataset/tfm/modelo_knn_animo.pkl')
joblib.dump(vectorizer, '/content/drive/MyDrive/dataset/tfm/vectorizer_animo.pkl')

# Guardar el DataFrame con la columna 'Animo'
df.to_csv('/content/drive/MyDrive/dataset/tfm/scraping_animeList_with_animo.csv', index=False)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


## Ejemplo de uso

In [7]:
# Cargar los modelos guardados
modelo_knn = joblib.load('/content/drive/MyDrive/dataset/tfm/modelo_knn_animo.pkl')
vectorizer = joblib.load('/content/drive/MyDrive/dataset/tfm/vectorizer_animo.pkl')

"""
  Recomiendar animes basados en el estado de ánimo proporcionado.
  Se selecciona un anime aleatorio del estado de ánimo dado y se encuentran
  los animes más similares usando KNN.
"""
def recomendar_animes(estado):
    animes = df[df['Animo'] == estado]
    if animes.empty:
        return []

    random_anime = animes.sample(1)
    anime_index = random_anime.index[0]

    X_tfidf_anime = vectorizer.transform([df['Synopsis'][anime_index]])
    distances, indices = modelo_knn.kneighbors(X_tfidf_anime)

    recomendados = df.iloc[indices[0]]['Title'].values
    return recomendados

# Comprobar recomendaciones para "feliz" o "triste" o "neutral"
print(recomendar_animes('neutral'))


['YouTuNya' 'Himawari Girls Mini Anime' 'Magic Door'
 'OTMGirls no Yokiyoki Channel'
 'Hataraku Saibou: Necchuushou - Moshimo Pocari Sweat ga Attara']
